# Improve RAG Systems with User Feedback

In this example notebook, we showcase how to collect user feedback with the LastMile AI Tracing SDK and improve your RAG systems using this feedback via the RAG Debugger tool.


## Notebook Outline
* [Introduction](#intro)
* [Setup](#setup)
* [Part 1: Log User Feedback](#part1)
  * [Trace your code](#trace_code)
  * [Annotate trace with user feedback](#annotate)
* [Part 2: Analyze User Feedback](#part2)
  * [View user feedback results](#view)
  * [Filter traces based on user feedback](#filter)
  * [Aggregate user feedback](#aggregate)
* [More Resources](#more)



<a name="intro"></a>
# Introduction
**User Feedback** is the feedback responses obtained from end users. These responses are typically related to the quality of the output of your application.

RAG Debugger allows you to incorporate user feedback into your trace via the LastMile Tracing SDK, providing valuable insights into how users perceive the performance your RAG system.

User feedback can be captured in various forms:

* **Feedback Score:** Users can rate a specific span, such as a LLM-generated output or a retrieved document (e.g., 👍 / 👎 or on a scale 1-10)
* **Expected Value:** Users can provide the expected or correct output for a span, which is stored in the expected field alongside input and output. This is useful for capturing corrections.
* **User Comment:** Users can share additional context or qualitative feedback using a free-form text field.

<a name="setup"></a>
# Setup

To begin, we need to install lastmile-eval library. We are using ipywidgets for this example so installing that here too.

In [ ]:
!pip install lastmile-eval --upgrade
!pip install ipywidgets

Import all modules used in this tutorial.

In [1]:
import os
from lastmile_eval.rag.debugger.tracing import get_lastmile_tracer
from lastmile_eval.rag.debugger.api import LastMileTracer
from lastmile_eval.rag.debugger.tracing.sdk import get_span_id, get_trace_id
from openai import OpenAI
from IPython.display import display, clear_output
from ipywidgets import Button, HBox, Output

Before we start this tutorial, we need the following tokens/keys:

* LastMile AI API Token: Go to the [LastMile Settings page](https://lastmileai.dev/settings?page=tokens). You will need to first create a LastMile AI account.
* OpenAI API Key: Go to [OpenAI API Keys page](https://platform.openai.com/account/api-keys) to create and access your OpenAI API Key.

We're using Google Colab's Secret Manager to set our tokens in this notebook.

In [2]:
# Run this cell if using google colab
try:
    from google.colab import userdata
    os.environ['OPENAI_API_KEY'] =  userdata.get('OPENAI_API_KEY')
    os.environ['LASTMILE_API_TOKEN'] =  userdata.get('LASTMILE_API_TOKEN')
except:
    pass

<a name="part1"></a>

# Part 1: Log User Feedback
In this example, we generate a riddle using `gpt-3.5-turbo`. We trace our code with the LastMile AI Tracing SDK.

We collect user feedback on the quality of the riddle (👍/👎) and annotate this feedback to our trace.

<a name="trace_code"></a>

## Trace your code
First, instantiate a LastMile Tracer object.

The name of the `Tracer` is also your Project Name that will be visible in the RAG Debugger UI.

In [3]:
tracer: LastMileTracer = get_lastmile_tracer(
    tracer_name="riddle-generator",
    lastmile_api_token= os.environ['LASTMILE_API_TOKEN'],
)

2024-05-29 13:32:25,502 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 13:32:25,644 - https://lastmileai.dev:443 "GET /api/evaluation_projects/list?name=riddle-generator HTTP/1.1" 200 351


The function generates our riddle using `gpt-3.5-turbo`.

In [ ]:
# Decorate function with tracer
@tracer.start_as_current_span("generate_riddle")
def generate()-> str:
    """
    Generates a riddle using OpenAI's GPT-3.5-turbo model.
    """
    response = OpenAI().chat.completions.create(messages = [{"role": "user", "content":"tell me a riddle"}], model = "gpt-3.5-turbo")
    riddle = response.choices[0].message.content

    # Save span and trace IDs of generated riddle (key = riddle)
    # Use these IDs to link user feedback to a specific trace / span
    tracer.store_trace_id(key=riddle, span_id=get_span_id(), trace_id= get_trace_id())

    return riddle

riddle = generate()

<a name="annotate"></a>

## Annotate trace with user feedback

The `get_and_store_user_feedback` function below generates a UI element to give a thumbs up/down.

We log the result (thumbs up/down reaction) back to the Trace and Span using the trace_id and span_id of our `generate` function.

In [ ]:
import trace


output = Output()

# Set trace_id and span_id so we can link user feedback to them
trace_id, span_id = tracer.read_trace_id(key=riddle)

def get_and_store_user_feedback():
    def on_thumbsup_clicked(b):
        with output:
            clear_output()
            print("You selected: 👍")
            tracer.log_binary_feedback(value=True, trace_id=trace_id)
            # Alternatively, you can log feedback using a string
            # tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="👍")

    def on_thumbsdown_clicked(b):
        with output:
            clear_output()
            print("You selected: 👎")
            tracer.log_binary_feedback(value=False, trace_id=trace_id)
            # tracer.log_feedback(span_id=span_id, trace_id=trace_id, feedback="👎")

    thumbsup_button = Button(description="👍", button_style="success")
    thumbsup_button.on_click(on_thumbsup_clicked)

    thumbsdown_button = Button(description="👎", button_style="danger")
    thumbsdown_button.on_click(on_thumbsdown_clicked)

    display(HBox([thumbsup_button, thumbsdown_button]))
    display(output)

print(f'\n Generated Riddle:\n{riddle}\n Give it a thumbs up or thumbs down to provide feedback.\n')
get_and_store_user_feedback()

2024-05-29 13:32:49,522 - Starting new HTTPS connection (1): lastmileai.dev:443
2024-05-29 13:32:49,607 - https://lastmileai.dev:443 "GET /api/evaluation_key_value_store/read?projectId=clwqs1yrd009zpb6nxmvqtkyw&key=I+have+keys+but+can%27t+open+any+locks.+I+have+a+space+but+no+room.+You+can+enter%2C+but+can%27t+go+outside.+What+am+I%3F HTTP/1.1" 200 385


{'createdAt': '2024-05-29T17:32:26.793Z', 'updatedAt': '2024-05-29T17:32:26.793Z', 'creatorId': 'clkrgxm850004phi6ee5mvhd1', 'evaluationProjectId': 'clwqs1yrd009zpb6nxmvqtkyw', 'key': "I have keys but can't open any locks. I have a space but no room. You can enter, but can't go outside. What am I?", 'value': '1352465596163432677', 'metadata': {'trace_id': '7294420575201929150837001412034271782'}}

 Generated Riddle:
I have keys but can't open any locks. I have a space but no room. You can enter, but can't go outside. What am I?
 Give it a thumbs up or thumbs down to provide feedback.



Output()

2024-05-29 13:32:54,835 - handle_msg[03e8070527bb46289db03fedc66af607]({'header': {'date': datetime.datetime(2024, 5, 29, 17, 32, 54, 831000, tzinfo=tzutc()), 'msg_id': 'c801a44b-6416-4add-b1e8-05dfc0c5cb23', 'msg_type': 'comm_msg', 'session': '7e7e4418-8c65-4bfa-95b5-436c7e646977', 'username': '495d3de3-0872-449f-b03d-9412cf776c9e', 'version': '5.2'}, 'msg_id': 'c801a44b-6416-4add-b1e8-05dfc0c5cb23', 'msg_type': 'comm_msg', 'parent_header': {}, 'metadata': {}, 'content': {'comm_id': '03e8070527bb46289db03fedc66af607', 'data': {'method': 'custom', 'content': {'event': 'click'}}}, 'buffers': []})
2024-05-29 13:32:55,014 - handle_msg[f242f7e86f854c3e86261f4bf5e76fe5]({'header': {'date': datetime.datetime(2024, 5, 29, 17, 32, 54, 862000, tzinfo=tzutc()), 'msg_id': 'f99a50dc-4f21-4d75-8786-b6ffd45572b8', 'msg_type': 'comm_msg', 'session': '7e7e4418-8c65-4bfa-95b5-436c7e646977', 'username': '495d3de3-0872-449f-b03d-9412cf776c9e', 'version': '5.2'}, 'msg_id': 'f99a50dc-4f21-4d75-8786-b6ffd45

Now our user feedback has been logged to our Trace!

<a name="view"></a>
## View user feedback results
User feedback is 1:1 with a specific trace. You can view user feedback in the 'Traces' page of RAG Debugger.

#### From your terminal:

Export your LASTMILE_API_TOKEN

```bash
export LASTMILE_API_TOKEN="<your-api-token>"
```

Run this CLI command to access the UI

```bash
rag-debug launch
```
Navigate to the 'Traces' Page where you see all the Traces listed. One of the columns here is 'User Feedback'.

<img width="973" alt="user_feedback" src="https://github.com/lastmile-ai/aiconfig/assets/81494782/0bf846d4-49d4-4319-960c-285725f94eb0"/>

It's helpful to have User Feedback here with other metadata associated with the Trace for easy debugging.


<a name="filter"></a>
## Filter traces based on user feedback

You can filter traces based on user feedback to debug your RAG system. For example, you might want to dig deeper into only the traces that have 'thumbs down' feedback.

Click the 'Filter' button on the 'Traces' page. Here you have advanced filtering options to look at Traces that yielded specific user feedback.

With the filtered set of traces, you can:
* Look for specific patterns in these traces (ex. paramSet)
* Create a TestSet from these traces. You can use these traces to experiment with changes to your RAG system.

<a name="aggregate"></a>

## Aggregate user feedback

**Project-level**

LastMile will automatically average user feedback (if numeric) at the project-level.

**Selected traces**

You have the option to select and download traces as a .csv file. This not only provides easy access to your data but also offers the flexibility to aggregate / transform user feedback according to your preferences.




<a name="more"></a>

# More Resources
* [RAG Debugger Documentation](https://rag.lastmileai.dev/docs/basics)
* [Discord - ask us anything!](https://discord.com/invite/xBhNKTetGx)